# Фино подешавање Open AI модела

Овај бележник је заснован на тренутним упутствима из [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) документације од Open AI.

Фино подешавање побољшава перформансе основних модела за вашу апликацију тако што се модел поново тренира са додатним подацима и контекстом који су релевантни за одређени случај или сценарио. Имајте у виду да технике као што су _few shot learning_ и _retrieval augmented generation_ омогућавају да обогатите основни упит релевантним подацима ради побољшања квалитета. Међутим, ови приступи су ограничени максималном величином прозора за токене код изабраног основног модела.

Са финим подешавањем, ми заправо поново тренирамо сам модел са потребним подацима (што нам омогућава да користимо много више примера него што може да стане у максимални прозор за токене) – и постављамо _прилагођену_ верзију модела којој више није потребно да има примере у тренутку извођења. Ово не само да побољшава ефикасност нашег дизајна упита (имамо већу флексибилност у коришћењу прозора за токене за друге ствари), већ потенцијално смањује и трошкове (смањујући број токена које морамо да пошаљемо моделу у тренутку извођења).

Фино подешавање има 4 корака:
1. Припремите податке за тренирање и отпремите их.
1. Покрените посао тренирања да добијете фино подешен модел.
1. Оцените фино подешен модел и поновите процес ради квалитета.
1. Поставите фино подешен модел за извођење када будете задовољни.

Имајте у виду да не подржавају сви основни модели фино подешавање – [погледајте OpenAI документацију](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) за најновије информације. Такође можете фино подесити модел који је већ био фино подешаван. У овом туторијалу користићемо `gpt-35-turbo` као наш основни модел за фино подешавање.

---


### Корак 1.1: Припремите свој скуп података

Хајде да направимо чет-бота који вам помаже да разумете периодни систем елемената тако што одговара на питања о неком елементу у виду лимерика. У _овом_ једноставном туторијалу, направићемо скуп података за тренирање модела са неколико примера одговора који показују очекивани формат података. У стварној примени, морали бисте да направите скуп података са много више примера. Можда ћете моћи да искористите и неки отворени скуп података (за вашу област) ако постоји, и да га прилагодите за фино подешавање.

Пошто се фокусирамо на `gpt-35-turbo` и желимо одговор у једном кораку (chat completion), можемо направити примере користећи [овај предложени формат](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) који одговара OpenAI захтевима за chat completion. Ако очекујете разговор у више корака, користили бисте [формат за више корака](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) који укључује параметар `weight` да означи које поруке треба (или не треба) користити у процесу фино подешавања.

Овде ћемо користити једноставнији формат са једним кораком. Подаци су у [jsonl формату](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) са једним записом по линији, где је сваки запис представљен као JSON објекат. Испод је приказано 2 записа као пример – погледајте [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) за комплетан скуп примера (10 примера) који ћемо користити у овом туторијалу за фино подешавање. **Напомена:** Сваки запис _мора_ бити дефинисан у једној линији (не раздвајајте га у више редова као што је уобичајено у форматираном JSON фајлу)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

У стварној примени биће вам потребан много већи скуп примера за добре резултате – баланс је између квалитета одговора и времена/трошкова за фино подешавање. Користимо мали скуп да бисмо брзо завршили фино подешавање и илустровали процес. Погледајте [овај пример из OpenAI Cookbook-а](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) за сложенији туторијал о финој обуци.


### Корак 1.2 Отпремите свој скуп података

Отпремите податке користећи Files API [како је описано овде](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Имајте у виду да, да бисте покренули овај код, прво морате да урадите следеће кораке:
 - Инсталирали сте `openai` Python пакет (проверите да користите верзију >=0.28.0 ради најновијих функција)
 - Поставили сте `OPENAI_API_KEY` променљиву окружења на ваш OpenAI API кључ
За више информација, погледајте [Упутство за подешавање](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) које је обезбеђено за овај курс.

Сада покрените код да бисте креирали фајл за отпремање из вашег локалног JSONL фајла.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### Корак 2.1: Креирајте посао за фино подешавање помоћу SDK-a


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### Корак 2.2: Проверите статус посла

Ево неколико ствари које можете урадити помоћу `client.fine_tuning.jobs` API-ја:
- `client.fine_tuning.jobs.list(limit=<n>)` - Прикажи последњих n послова за фино подешавање
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Добиј детаље о одређеном послу за фино подешавање
- `client.fine_tuning.jobs.cancel(<job_id>)` - Откажи посао за фино подешавање
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Прикажи до n догађаја са посла
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Први корак у овом процесу је _валидација фајла за тренирање_ како би се осигурало да су подаци у исправном формату.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### Корак 2.3: Пратите догађаје да бисте пратили напредак


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Корак 2.4: Погледајте статус на OpenAI контролној табли


Можете такође да видите статус тако што ћете посетити OpenAI веб сајт и истражити одељак _Fine-tuning_ на платформи. Овде можете видети статус тренутног посла, као и пратити историју претходних извршавања. На овој слици екрана, можете видети да је претходно извршавање било неуспешно, а да је друго покретање било успешно. За контекст, ово се догодило када је прво покретање користило JSON фајл са неправилно форматираним записима – када је то исправљено, друго покретање је успешно завршено и модел је постао доступан за коришћење.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.sr.png)


Можете такође да видите статусне поруке и метрике тако што ћете скроловати ниже у визуелној контролној табли као што је приказано:

| Поруке | Метрике |
|:---|:---|
| ![Поруке](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.sr.png) |  ![Метрике](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.sr.png)|


### Корак 3.1: Преузмите ID и тестирајте фино подешени модел у коду


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### Корак 3.2: Учитајте и тестирајте фино подешени модел у Playground-у

Сада можете тестирати фино подешени модел на два начина. Прво, можете посетити Playground и користити падајући мени Models да изаберете свој нови фино подешени модел из понуђених опција. Друга опција је да користите "Playground" опцију која се приказује у панелу за фино подешавање (погледајте слику изнад), што покреће _упоредни_ приказ у коме се основни и фино подешени модел приказују један поред другог ради брзе процене.

Само попуните системски контекст који сте користили у својим подацима за обуку и унесите своје тест питање. Приметићете да се обе стране ажурирају са истим контекстом и питањем. Покрените поређење и видећете разлику у излазима између њих. _Обратите пажњу како фино подешени модел приказује одговор у формату који сте навели у својим примерима, док основни модел само прати системски упит._

Приметићете и да поређење приказује број токена за сваки модел, као и време потребно за инференцију. **Овај конкретан пример је поједностављен и служи само да прикаже процес, али не одражава стварни скуп података или сценарио из праксе.** Можда ћете приметити да оба примера имају исти број токена (системски контекст и кориснички упит су идентични), али да фино подешени модел троши више времена на инференцију (прилагођени модел).

У стварним ситуацијама нећете користити овако једноставан пример, већ ћете фино подешавати модел на стварним подацима (нпр. каталог производа за корисничку подршку), где ће квалитет одговора бити много очигледнији. У _том_ контексту, да бисте добили једнак квалитет одговора са основним моделом, биће потребно више прилагођавања упита, што ће повећати потрошњу токена и потенцијално време обраде за инференцију. _Ако желите да испробате ово, погледајте примере фино подешавања у OpenAI Cookbook-у за почетак._



---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем AI услуге за превођење [Co-op Translator](https://github.com/Azure/co-op-translator). Иако настојимо да обезбедимо тачност, имајте у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на изворном језику треба сматрати меродавним извором. За критичне информације препоручује се професионални људски превод. Не сносимо одговорност за било каква неспоразума или погрешна тумачења настала коришћењем овог превода.
